In [45]:
import pdfplumber
import pandas as pd
import re

#Funcionário:
#Avos 13o:
#Sal:
#Adiantamento:
#1a.Parcela:
#Provisão 13o:
#Salário Refer:
#FGTS:
#INSS:

def extract_text_from_pdf(pdf_path):
    all_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf.pages)):
            text = pdf.pages[page_num].extract_text()
            print(f'Verificando página: {page_num + 1}...')

            funcionario_indices = [m.start() for m in re.finditer("Funcionário:", text)]

            for index in funcionario_indices:
                data = process_text(text[index:])
                all_data.append(data)

    return all_data

def has_information(text):
    keywords = ["Funcionário:"]
    
    for keyword in keywords:
        if keyword in text:
            return True
    
    return False

def get_value_by_funcionario(text):
    funcionario_id, funcionario_nome = "", ""
    keyword = "Funcionário:"
    
    if keyword in text:
        start_index = text.find(keyword) + len(keyword)
        funcionario_info = text[start_index:text.find('\n', start_index)].strip()
        
        # Remover "Avos 13o.:" e tudo que vier depois
        funcionario_info = re.sub(r'Avos 13o\..*$', '', funcionario_info)

        if "Funcionário: " in funcionario_info:
            funcionario_info = funcionario_info.replace("Funcionário: ", "")
        
        funcionario_split = funcionario_info.split('-')
        
        if len(funcionario_split) == 2:
            funcionario_id, funcionario_nome = map(str.strip, funcionario_split)
    
    return funcionario_id, funcionario_nome


def get_value_by_avos13o(text):
    indice_avos = text.find("Avos 13o.:")
    
    # Verificar se a palavra foi encontrada
    if indice_avos == -1:
        return None
    
    # Obter o texto após a palavra
    texto_apos_avos = text[indice_avos + len("Avos 13o.:"):].strip()

    # Dividir o texto em um array usando espaços como delimitador
    array_resultante = texto_apos_avos.split(" ")

    # Pegar apenas o primeiro item do array
    primeiro_item = array_resultante[0]

    # Extrair apenas os números usando regex
    numeros = re.findall(r'\d+', primeiro_item)

    # Se houver números, pegar o primeiro, caso contrário, retornar None
    avos13o_value = numeros[0] if numeros else None

    return avos13o_value

def get_value_by_sal(text):
    indice_sal = text.find("Sal:")
    
    # Verificar se a palavra foi encontrada
    if indice_sal == -1:
        return None
    
    # Obter o texto após a palavra
    texto_apos_sal = text[indice_sal + len("Sal:"):].strip()

    # Encontrar a primeira palavra (conjunto de caracteres antes de um espaço)
    sal_match = re.match(r'\S+', texto_apos_sal)

    sal_value = sal_match.group() if sal_match else None

    return sal_value

def get_value_by_adiantamento(text):
    keyword = "1a.Parcela..:"
    adiantamento_value = []

    if keyword in text:
        start_index = text.find(keyword) + len(keyword)

        # Encontrar os 10 primeiros conjuntos de caracteres após a palavra-chave
        for _ in range(10):
            start_index = text.find('\n', start_index) + 1

            # Encontrar o próximo conjunto de caracteres até o próximo espaço
            value_match = re.match(r'\S+', text[start_index:])
            value = value_match.group() if value_match else None
            adiantamento_value.append(value)

    return adiantamento_value

def get_value_by_primeira_parcela(text):
    keyword = "1a.Parcela..:"
    primeira_parcela_value = ""

    if keyword in text:
        start_index = text.find(keyword) + len(keyword)

        for _ in range(2):
            start_index = text.find('\n', start_index) + 1
        
        end_index = text.find('\n', start_index)

        primeira_parcela_value = text[start_index:end_index].strip()

    return primeira_parcela_value

def get_value_by_provisao_13o(text):
    keyword = "Provisão 13o:"
    provisao_13o_value = ""

    if keyword in text:
        start_index = text.find(keyword) + len(keyword)

        # Encontrar todos os conjuntos de caracteres após a palavra-chave
        conjuntos = re.findall(r'\S+', text[start_index:])

        # Pegar apenas o primeiro conjunto
        provisao_13o_value = conjuntos[0] if conjuntos else None

    return provisao_13o_value

def get_value_by_salario_refer(text):
    keyword = "Salário Refer..:"
    salario_refer_value = ""

    if keyword in text:
        start_index = text.find(keyword) + len(keyword)
        conjuntos = re.findall(r'\S+', text[start_index:])

        salario_refer_value = conjuntos[0] if conjuntos else None
    return salario_refer_value
    
def get_value_by_fgts(text):
    keyword = "FGTS"
    fgts_value = ""

    if keyword in text:
        start_index = text.find(keyword) + len(keyword)
        conjuntos = re.findall(r'\S+', text[start_index:])

        fgts_value = conjuntos[2] if conjuntos else None
    return fgts_value

def get_value_by_inss(text):
    keyword = "INSS"
    inss_value = ""

    if keyword in text:
        start_index = text.find(keyword) + len(keyword)
        conjuntos = re.findall(r'\S+', text[start_index:])

        inss_value = conjuntos[2] if conjuntos else None
    return inss_value

def process_text(text):
    if not has_information(text):
        return None
    
    funcionario_id, funcionario_nome = get_value_by_funcionario(text)
    avos13o_value = get_value_by_avos13o(text)
    sal_value = get_value_by_sal(text)
    adiantamento_value = get_value_by_keyword(text, "Adiantamento:")
    primeira_parcela_value = get_value_by_keyword(text, "1a.Parcela..:")
    provisao_13o_value = get_value_by_provisao_13o(text)
    salario_refer_value = get_value_by_salario_refer(text)
    fgts_value = get_value_by_fgts(text)
    inss_value = get_value_by_inss(text)

    print(f"Funcionário ID: {funcionario_id}, Nome: {funcionario_nome}, AVOS13o: {avos13o_value}, Sal: {sal_value}, Adiantamento: {adiantamento_value}, Primeira Parcela: {primeira_parcela_value}, Provisão 13o: {provisao_13o_value}, Salário Refer: {salario_refer_value}, FGTS: {fgts_value}, INSS {inss_value}")

    data = {
        "ID": funcionario_id,
        "NOME": funcionario_nome,
        "AVOS 13o": avos13o_value,
        "SAL": sal_value,
        "ADIANTAMENTO": adiantamento_value,
        "PRIMERA PARCELA": primeira_parcela_value,
        "PROVISAO 13o": provisao_13o_value,
        "SALÁRIO REFER": salario_refer_value,
        "FGTS": fgts_value,
        "INSS":inss_value
    }
    return data

def get_value_by_keyword(text, keyword, end_marker=None):
    value = ""
    
    if keyword in text:
        start_index = text.find(keyword) + len(keyword)

        if end_marker:
            for _ in range(1 if end_marker == "1a.Parcela..:" else 3):
                start_index = text.find('\n', start_index) + 1

            end_index = text.find('\n', start_index) if end_marker == "1a.Parcela..:" else text.find(end_marker, start_index)
            value = text[start_index:end_index].replace("1a.Parcela..:", "").replace(" 0,00 2a.Parc.Proc:", "").strip()
        else:
            end_index = text.find('\n', start_index)
            value = text[start_index:end_index].strip()

    return value

pdf_path = r'D:\1Desktop\Documentos\My Web Sites\App py\Prototipo14\PROVISAO 13 SALARIO COMPLETA 112023 PDF.pdf'
excel_path = r'D:\1Desktop\Documentos\My Web Sites\App py\Prototipo14\Provisão Salário.xlsx'
all_data = extract_text_from_pdf(pdf_path)
df = pd.DataFrame(all_data)
df.to_excel(excel_path, index=False)
print(f'ARQUIVO SALVO EM {excel_path}')


Verificando página: 1...
Funcionário ID: 01454, Nome: BARBARA REGIANE MADUREIRA BELLO DA SILVA, AVOS13o: 11, Sal: 12.537,61, Adiantamento: 0,00, Primeira Parcela: 6.145,96, Provisão 13o: 5.346,85, Salário Refer: 12.537,61, FGTS: 427,75, INSS 3.368,89
Funcionário ID: 01570, Nome: OLIVER SUADICANI, AVOS13o: 11, Sal: 57.282,94, Adiantamento: 0,00, Primeira Parcela: 28.641,47, Provisão 13o: 23.867,89, Salário Refer: 57.282,94, FGTS: 1.909,43, INSS 15.392,07
Funcionário ID: 01701, Nome: ARMIN KOLLMANNTHALER, AVOS13o: 11, Sal: 12.537,61, Adiantamento: 0,00, Primeira Parcela: 6.145,96, Provisão 13o: 5.412,43, Salário Refer: 12.609,15, FGTS: 432,99, INSS 3.388,11
Funcionário ID: 01769, Nome: CATHERINE DE MORAIS LEBECK NETTO, AVOS13o: 5, Sal: 4.620,27, Adiantamento: 0,00, Primeira Parcela: 1.171,28, Provisão 13o: 797,87, Salário Refer: 4.725,95, FGTS: 63,83, INSS 577,22
Verificando página: 2...
Funcionário ID: 01414, Nome: CARLOS EDUARDO GONCALEZ, AVOS13o: 11, Sal: 14.459,87, Adiantamento: 0,00